# Общий уровень самоубийств (на 100 000 населения)

In [1]:
import pandas as pd

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке \
https://www.who.int/data/gho/data/themes/mental-health/suicide-rates

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_suicide_mortality.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,MH_12,Age-standardized suicide rates (per 100 000 po...,numeric,AMR,Americas,Country,BRB,Barbados,Year,2019,...,NaN,NaN,0.22,NaN,0.42,0.3 [0.2-0.4],NaN,NaN,EN,2021-02-08T21:00:00.000Z
1,MH_12,Age-standardized suicide rates (per 100 000 po...,numeric,AMR,Americas,Country,ATG,Antigua and Barbuda,Year,2019,...,NaN,NaN,0.22,NaN,0.45,0.3 [0.2-0.5],NaN,NaN,EN,2021-02-08T21:00:00.000Z


Описание нужных нам столбцов.

- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - стандартизированные по возрасту показатели самоубийств (на 100 000 населения)

Оставим только их.

In [3]:
data = data[['SpatialDimValueCode', 'Period', 'FactValueNumeric']]

# Переименуем столбец FactValueNumeric в SuicideMortality
data.rename(
    columns={'FactValueNumeric': 'SuicideMortality'},
    inplace=True,
)

data.head()

,SpatialDimValueCode,Period,SuicideMortality
0,BRB,2019,0.31
1,ATG,2019,0.32
2,GRD,2019,0.64
3,VCT,2019,1.01
4,NER,2019,10.15


In [4]:
# Убедимся, что нет пропусков
data.isna().sum().sum()

0

In [5]:
print(f"Данные представлены за {data['Period'].min()} - {data['Period'].max()} гг.")

Данные представлены за 2000 - 2019 гг.


Данные за 2020 г. заполним средним по стране за последние 3 года.

In [6]:
# Маска для последних 3-х лет
mask = data['Period'].isin([2019, 2018, 2017])

# Сгруппируем по коду страны и возьмем среднее
mean_last_3_years_data = data[mask].groupby('SpatialDimValueCode').agg('mean')

# Сбросим индекс
mean_last_3_years_data.reset_index(inplace=True)

# Проставим 2020 год
mean_last_3_years_data['Period'] = 2020

mean_last_3_years_data.head()

,SpatialDimValueCode,Period,SuicideMortality
0,AFG,2020,5.940000
1,AGO,2020,12.420000
2,ALB,2020,3.923333
3,ARE,2020,5.010000
4,ARG,2020,8.606667


Соединим данные за 2020 год с предыдущей таблицей.

In [7]:
data = pd.concat([
    data,
    mean_last_3_years_data,
], ignore_index=True)

data.head()

,SpatialDimValueCode,Period,SuicideMortality
0,BRB,2019,0.31
1,ATG,2019,0.32
2,GRD,2019,0.64
3,VCT,2019,1.01
4,NER,2019,10.15


In [8]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_suicide_mortality_prepared.csv', index=False)